In [1]:
import os
os.getcwd()
os.chdir("../..")
os.getcwd()

'/home/caleb/Cornell-Conversational-Analysis-Toolkit'

In [2]:
import convokit
import pickle
import pandas as pd


In [3]:
from convokit import Corpus, Conversation, Utterance

In [4]:
fake_news_dir = '/sauna/fake-news'
politics_dir = '/sauna/reddit_201810_raw/corpus/pokemontrades_banlist~-~politics/politics'
donald_dir = '/sauna/reddit_201810_raw/corpus/TheTwoBeerQueers~-~The_Donald/The_Donald/'

In [5]:
fake_news_urls = {'breitbart.com', 'dcgazette.com', "dailymail.co.uk", 
                  "lewrockwell.com", "newswars.com", "gellerreport.com",
                  "dcclothesline.com", "thegatewaypundit.com", "trueactivist.com"
                 }

In [6]:
real_news_sites = ['BBC', 'Business Insider', 'Buzzfeed', 'CBS News', 'CNN', 'Daily Beast', 
                   'FT Westminster Blog', 'FiveThirtyEight', 'Fortune', 'Mercury News', 'NPR',
                   'National Review', 'New Yorker', 'Newsweek', 'PBS', 'Politico', 'Real Clear Politics', 'Reuters',
                   'Slate', 'The American Conservative', 'The Denver Post', 'The Guardian', 'The Hill', 'The New York Times',
                   'The Verge', 'USA Today', 'Vox', 'WSJ Washington Wire', 'Washington Monthly', 'Washington Post']

In [7]:
real_news_urls = {'bbc.com', 'businessinsider.com', 'buzzfeed.com', 'cbsnews.com', 'cnn.com', 'thedailybeast.com',
                  'ft.com', 'fivethirtyeight.com', 'fortune.com', 'mercurynews.com', 'npr.org', 'nationalreview.com',
                  'newyorker.com', 'newsweek.com', 'pbs.org', 'politico.com', 'realclearpol', 'reuters.com',
                  'slate.com', 'theamericanconservative.com', 'denverpost.com', 'theguardian.com', 'thehill.com',
                  'nytimes.com', 'theverge.com', 'usatoday.com', 'vox.com', 'wsj.com', 'washingtonmonthly.com', 
                  'washingtonpost.com'
                 }

In [8]:
# politics_corpus = Corpus(filename=politics_dir)
politics_corpus = Corpus(filename=os.path.join(fake_news_dir, 'politics_corpus_paired'))
# donald_corpus = Corpus(filename=os.path.join(fake_news_dir, 'donald_corpus_valid_convos'))

In [9]:
# Start time: 01 Oct 2017
# End time: 01 Oct 2018
start_time = 1506816000
end_time = 1538352000

In [10]:
from collections import defaultdict
real_news_convos = defaultdict(set)
fake_news_convos = defaultdict(set)
for convo in politics_corpus.iter_conversations():
    if start_time <= convo.meta['timestamp'] <= end_time:
        if convo.meta['domain'] in real_news_urls:
            real_news_convos[convo.meta['domain']].add(convo.id)
        elif convo.meta['domain'] in fake_news_urls:
            fake_news_convos[convo.meta['domain']].add(convo.id)

In [11]:
for k, v in real_news_convos.items():
    print("{}: {}".format(k, len(v)))
    
print("Total posts:", sum([len(v) for v in real_news_convos.values()]))

washingtonpost.com: 136
thehill.com: 234
slate.com: 22
cnn.com: 74
thedailybeast.com: 57
reuters.com: 35
nytimes.com: 74
businessinsider.com: 59
politico.com: 59
cbsnews.com: 25
newsweek.com: 60
vox.com: 27
theamericanconservative.com: 3
theguardian.com: 32
bbc.com: 15
wsj.com: 8
usatoday.com: 21
buzzfeed.com: 13
newyorker.com: 4
fivethirtyeight.com: 2
npr.org: 14
pbs.org: 4
nationalreview.com: 6
denverpost.com: 1
theverge.com: 1
washingtonmonthly.com: 2
mercurynews.com: 1
fortune.com: 1
Total posts: 990


In [12]:
for k, v in fake_news_convos.items():
    print("{}: {}".format(k, len(v)))
    
print("Total posts:", sum([len(v) for v in fake_news_convos.values()]))

breitbart.com: 987
dailymail.co.uk: 3
Total posts: 990


In [13]:
from itertools import chain
real_news_convos_ids = list(chain.from_iterable(real_news_convos.values()))
fake_news_convos_ids = list(chain.from_iterable(fake_news_convos.values()))

In [14]:
valid_convos_ids = set(real_news_convos_ids + fake_news_convos_ids)

In [15]:

politics_corpus.print_summary_stats()

Number of Users: 59141
Number of Utterances: 278813
Number of Conversations: 1980


In [19]:
# politics_corpus.filter_conversations_by(func=lambda convo: convo.id in valid_convos_ids)

In [18]:
politics_corpus.print_summary_stats()

Number of Users: 59141
Number of Utterances: 278813
Number of Conversations: 1980


In [24]:
# politics_corpus.dump('politics_corpus_valid_convos', base_path=fake_news_dir)

In [19]:
thread_pfxs = politics_corpus.utterance_threads(prefix_len=8, include_root=False)
convos_with_valid_len = set()
convo_to_thread = defaultdict(list)
for thread_id, utts in thread_pfxs.items():
    if len(utts) >= 8:
        convos_with_valid_len.add(utts[next(iter(utts))].root)
        convo_to_thread[utts[next(iter(utts))].root].append(thread_id)

In [20]:
from random import choice

for convo, threads in convo_to_thread.items():
    convo_to_thread[convo] = choice(threads)

In [21]:
thread_to_convo = {v: k for k, v in convo_to_thread.items() if len(v) > 0}

In [22]:
real_news_thread_ids = [convo_to_thread[convo_id] for convo_id in real_news_convos_ids if len(convo_to_thread[convo_id]) > 0]
fake_news_thread_ids = [convo_to_thread[convo_id] for convo_id in fake_news_convos_ids if len(convo_to_thread[convo_id]) > 0]



In [23]:
len(real_news_thread_ids)

990

In [24]:
len(fake_news_thread_ids)

990

In [25]:
from random import sample
real_news_thread_sample = sample(real_news_thread_ids, len(fake_news_thread_ids))

In [26]:
real_news_convos_sample = [thread_to_convo[thread_id] for thread_id in real_news_thread_sample]

In [27]:
fake_news_long_convos = [thread_to_convo[thread_id] for thread_id in fake_news_thread_ids]

In [28]:
paired_convos = set(real_news_convos_sample).union(fake_news_long_convos)

In [29]:
balanced_threads = fake_news_thread_ids + real_news_thread_sample

In [85]:
len(set(balanced_threads))

1980

In [30]:
len(paired_convos)

1980

In [87]:
# politics_corpus.filter_conversations_by(func=lambda convo: convo.id in paired_convos)

In [89]:
# politics_corpus.dump('politics_corpus_paired', base_path=fake_news_dir)

In [ ]:
# donald_corpus = Corpus(filename=os.path.join(fake_news_dir, 'donald_corpus_paired'))

In [31]:
politics_corpus.print_summary_stats()

Number of Users: 59141
Number of Utterances: 278813
Number of Conversations: 1980


In [32]:
hc = convokit.HyperConvo(min_thread_len=8, prefix_len=8, include_root=False)

In [33]:
motif_feats = hc.retrieve_motif_counts(politics_corpus)

In [34]:
balanced_threads = set(balanced_threads)

In [35]:
motif_feats = {k: v for k, v in motif_feats.items() if k in balanced_threads}

In [36]:
motif_counts_df = pd.DataFrame.from_dict(motif_feats, orient='index')
motif_feat_names = list(motif_counts_df.columns)

In [37]:
hyperconv_feats = hc.retrieve_feats(politics_corpus)
hyperconv_feats = {k: v for k, v in hyperconv_feats.items() if k in balanced_threads}

/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:182: RuntimeWarning: invalid value encountered in long_scalars
  "norm.max": lambda l: np.max(l) / np.sum(l),
/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:187: RuntimeWarning: invalid value encountered in long_scalars
  if len(l) > 1 else np.nan,
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/caleb/miniconda/envs/venv/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2614: RuntimeWarning: invalid value encountered in true_divide
  pk = 1.0*pk / np.sum(pk, axis=0)
/home/caleb/Cornell-Conversational-Analysis-Toolkit/convokit/hyperconvo/hyperconvo.py:194: Runtim

In [38]:
hyperconv_df = pd.DataFrame.from_dict(hyperconv_feats, orient='index')
hyperconv_feat_names = list(hyperconv_df.columns)

In [39]:
path_feats = hc.retrieve_motif_pathway_stats(politics_corpus)
path_feats = {k: v for k, v in path_feats.items() if k in balanced_threads}

In [40]:
path_stats_df = pd.DataFrame.from_dict(path_feats, orient='index')
columns = ['PATH-'+', '.join(filter(lambda x: type(x) == str, col)).strip() for col in path_stats_df.columns.values]
path_stats_df.columns = columns
path_feat_names = list(path_stats_df.columns)

In [41]:
feats_df = pd.concat([hyperconv_df, path_stats_df, motif_counts_df], axis=1)

In [42]:
feats_df['volume'] = [len(set(utt.user.name for utt in thread_pfxs[thread_id].values())) for thread_id in feats_df.index]

In [43]:
fake_news_thread_ids = set(fake_news_thread_ids)
y = [int(thread_id in fake_news_thread_ids) for thread_id in list(feats_df.index)]

In [44]:
import numpy as np
y = np.array(y)

In [45]:
feats_df['y'] = y

In [46]:
feats_df.shape

(1980, 262)

In [47]:
feats_df_no_na = feats_df.dropna()

In [48]:
len(feats_df_no_na)

1907

In [49]:
sum(feats_df_no_na['y'])

933

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut 
from sklearn.model_selection import cross_val_score

for feature_set, name in [(hyperconv_feat_names, "hyperconvo"),
                        (["volume"], "volume"),
                        (hyperconv_feat_names + ["volume"], "hyperconv-volume"),
                        (motif_feat_names, "motifs"),
                        (motif_feat_names + ["volume"], "motifs-volume"),
                        (path_feat_names, "motifpaths"),
                        (path_feat_names + ["volume"], "motifpaths-volume"),
                        (hyperconv_feat_names + motif_feat_names, "hyperconv-motif"),
                        (hyperconv_feat_names + path_feat_names, "hyperconv-paths"),
                        (hyperconv_feat_names + motif_feat_names + path_feat_names, "hyperconvo-motifall"),
                       ]:
    clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])      
    loo = LeaveOneOut()
    X = np.array(feats_df_no_na[feature_set])
    y = np.array(feats_df_no_na['y'])
    scores = cross_val_score(clf, X, y, cv=loo)
    print("- {}, cv_accuracy: {:.4f}".format(name, scores.mean()))

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     clf.fit(X_train, y_train)
#     train_score = clf.score(X_train, y_train)
#     test_score = clf.score(X_test, y_test)
#     print("- {}, train: {:.4f}, test: {:.4f}".format(name, train_score, test_score))
#     print_extreme_coefs(clf, feature_set, num_features = 5)
    

- hyperconvo, cv_accuracy: 0.5905
- volume, cv_accuracy: 0.5338
- hyperconv-volume, cv_accuracy: 0.5915
- motifs, cv_accuracy: 0.5747
- motifs-volume, cv_accuracy: 0.5810
- motifpaths, cv_accuracy: 0.5994
- motifpaths-volume, cv_accuracy: 0.6009
- hyperconv-motif, cv_accuracy: 0.5863


In [55]:
thread_pfxs = politics_corpus.utterance_threads(prefix_len=8, include_root=False)

## BoW prediction

In [56]:
thread_pfx_texts = {k: " ".join([utt.text for utt in v.values()]) for k, v in thread_pfxs.items() if k in balanced_threads}

In [75]:
y = [int(thread_id in real_news_thread_sample) for thread_id in list(feats_df.index)]

In [76]:
train_ids, test_ids, y_train, y_test = train_test_split(list(balanced_threads), y, test_size=0.2, random_state=42)

In [96]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(min_df=0.05, max_df=0.7, ngram_range=(1,3))

In [97]:
X_train = cv.fit_transform([thread_pfx_texts[train_id] for train_id in train_ids]).toarray()

In [98]:
X_test = cv.transform([thread_pfx_texts[test_id] for test_id in test_ids]).toarray()

In [99]:
clf = Pipeline([("standardScaler", StandardScaler()), ("logreg", LogisticRegression(solver='liblinear'))])      

clf.fit(X_train, y_train)
train_score = clf.score(X_train, y_train)
test_score = clf.score(X_test, y_test)
print("- {}, train: {:.4f}, test: {:.4f}".format("BoW", train_score, test_score))


- BoW, train: 0.7740, test: 0.5404


In [100]:
def print_extreme_coefs(clf, feature_names, num_features: int = 5):
    coefs = clf.named_steps['logreg'].coef_[0].tolist()

    assert len(feature_names) == len(coefs)

    feats_coefs = sorted(list(zip(feature_names, coefs)), key=lambda x: x[1], reverse=True)

    print()
    print("TOP {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[:num_features]:
        print("{}: {:.3f}".format(ft, coef))
    print()
    print("BOTTOM {} FEATURES".format(num_features))
    for ft, coef in feats_coefs[-num_features:]:
        print("{}: {:.3f}".format(ft, coef))
    print()

In [101]:
print_extreme_coefs(clf, cv.get_feature_names(), num_features = 10)


TOP 10 FEATURES
right wing: 1.076
https: 0.770
http: 0.581
www: 0.537
of the: 0.446
the right: 0.432
going: 0.406
years: 0.375
on: 0.371
said: 0.343

BOTTOM 10 FEATURES
org: -0.386
against: -0.395
talking about: -0.399
https www: -0.401
world: -0.407
do with: -0.422
right: -0.492
going to: -0.532
com: -0.815
wing: -1.256

